In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

# 1. Load the data

In [2]:
data = pd.read_csv('digit-recognizer/train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data.shape

(42000, 785)

In [5]:
data = np.array(data)

In [6]:
row, col = data.shape

# Shuffle the data
np.random.shuffle(data)

# 2. Split the data into Train & Val

In [7]:
# Split validation data and transpose it
data_val = data[0:1000].T

Y_val = data_val[0]
X_val = data_val[1:col]

In [8]:
# Split training data and transpose it
train_data = data[1000:row].T
Y_train = train_data[0]
X_train = train_data[1:col]

In [12]:
print("X_train shape: ", X_train.shape)

X_train shape:  (784, 41000)


# 3. Constructing the Neural Network Model

In [13]:
# Initialize parameters
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    
    return W1, b1, W2, b2

In [14]:
# Rectified Linear Unit Function (ReLU)
def ReLU(Z):
    return np.maximum(0, Z)

In [15]:
# Softmax Activation Function
def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

In [16]:
# Forward propagation
def forward(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    
    return Z1, A1, Z2, A2

In [17]:
# function to One Hot Encode
def one_hot_encode(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    
    return one_hot_Y

In [ ]:
# Derivative of ReLU
def deriv_ReLU(Z):
    return Z > 0

In [20]:
# Backward propagation
def backward(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    
    one_hot_Y = one_hot_encode(Y)
    
    # Layer [2] - Output Layer
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1/ m * np.sum(dZ2)
    
    # Layer [1] - Hidden Layer
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ2.dot(A1.T)
    db1 = 1/ m * np.sum(dZ2)
    
    return dW1, db1, dW2, db2

In [22]:
# Update Parameters
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha): # alpha => learning rate
    W1 = W1 - alpha * dW1
    b1 = W1 - alpha * db1
    W2 = W1 - alpha * dW2
    b2 = W1 - alpha * db2
    
    return W1, b1, W2, b2

In [23]:
# Predictions
def get_predictions(A2):
    return np.argmax(A2, 0)

In [24]:
# Accuracy
def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

In [25]:
# Gradient Descent
def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward(Z1, A1, Z2, A2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
        if i % 50 == 0:
            print("Iteration: ", i)
            print("Accuracy: ", get_accuracy(get_predictions(A2), Y))
            
    return W1, b1, W1, b2